In [1]:
import pandas as pd
import numpy as np
#Вероятно надо оставить только OrderedDict
from collections import OrderedDict

In [2]:
df = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None, names=['views', 'buys'])
df_test = pd.read_csv('coursera_sessions_test.txt', sep=';', header=None, names=['views', 'buys'])

In [3]:
df_test.tail()

,views,buys
49995,"37637,18363,18131",NaN
49996,"35402,10150,35402,10150,35402,88962,49526,3540...",NaN
49997,"161,1345,161,1345,161,1345,161,30853,161",NaN
49998,"26236,16572,26236",NaN
49999,"80626,102807,102808,102809",NaN


In [4]:
df_list = df.apply(lambda x: x.str.split(','), axis=0)
df_test_list = df_test.apply(lambda x: x.str.split(','), axis=0)

In [5]:
df_test_list

,views,buys
0,"[6, 7, 8]",NaN
1,"[13, 14, 15]",NaN
2,"[22, 23]",NaN
3,"[28, 29, 30, 31, 32, 33]",NaN
4,"[40, 41]",NaN
...,...,...
49995,"[37637, 18363, 18131]",NaN
49996,"[35402, 10150, 35402, 10150, 35402, 88962, 495...",NaN
49997,"[161, 1345, 161, 1345, 161, 1345, 161, 30853, ...",NaN
49998,"[26236, 16572, 26236]",NaN


In [6]:
v = np.hstack(df_list.views.to_numpy()).astype(np.int)
b = np.hstack(df_list.buys.dropna().to_numpy()).astype(np.int)

In [9]:
goods_dict = {}
for good in v:
    if good in goods_dict.keys():
        goods_dict[good] +=1
    else:
        goods_dict[good] =1

l = len(v) + 1

for key in goods_dict.keys():
    goods_dict[key] += (1 - np.where(v==key)[0][0]/l)

ordered_dict_goods = OrderedDict(sorted(goods_dict.items(), key=lambda x: x[1], reverse=True))
#ordered_dict_goods

In [10]:
buys_dict = {}
for buy in b:
    if buy in buys_dict.keys():
        buys_dict[buy] +=1
    else:
        buys_dict[buy] =1

l = len(b) + 1

for key in buys_dict.keys():
    buys_dict[key] += (1 - np.where(b==key)[0][0]/l)

ordered_dict_buys = OrderedDict(sorted(buys_dict.items(), key=lambda x: x[1], reverse=True))
#ordered_dict_buys

Надо это все в одну функцию объединить

In [109]:
def get_recomndations(list_of_views, type_rec=0):
    recomended_goods = {}
    buy_goods = {}
    for i in list_of_views:
        if i in ordered_dict_goods:
            recomended_goods[i] = ordered_dict_goods[i]
        else:
            recomended_goods[i] = 0
        if i in ordered_dict_buys:
            buy_goods[i] = ordered_dict_buys[i]
        else:
            buy_goods[i] = 0

    #print(OrderedDict(sorted(recomended_goods.items(), key=lambda x: x[1], reverse=True)))
    #print(OrderedDict(sorted(buy_goods.items(), key=lambda x: x[1], reverse=True)))
    if type_rec == 0:
        recomended_by_views = []
        for item in OrderedDict(sorted(recomended_goods.items(), key=lambda x: x[1], reverse=True)):
            if len(recomended_by_views) < 5:
                recomended_by_views.append(item)
            else:
                break
        return recomended_by_views
    
    recomended_by_buys = []
    for item in OrderedDict(sorted(buy_goods.items(), key=lambda x: x[1], reverse=True)):
        if len(recomended_by_buys) < 5:
            recomended_by_buys.append(item)
        else:
            break
    return recomended_by_buys

In [110]:
df_list['recomend_view'] = df_list.views.apply(lambda x: get_recomndations(list(map(int, x)), type_rec=0))
df_list['recomend_buy'] = df_list.views.apply(lambda x: get_recomndations(list(map(int, x)), type_rec=1))

In [111]:
df_list.dropna()

,views,buys,recomend_view,recomend_buy
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]","[63, 64, 60, 61, 65]","[67, 60, 63, 59, 61]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],"[85, 93, 89, 90, 84]","[86, 85, 93, 84, 87]"
19,"[138, 198, 199, 127]",[199],"[127, 138, 198, 199]","[199, 138, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],"[303, 306, 304, 307, 309]","[303, 304, 305, 306, 307]"
33,"[352, 353, 352]",[352],"[352, 353]","[352, 353]"
...,...,...,...,...
49943,"[41795, 4337, 4335, 4337, 4335, 24087, 4335, 4...",[4335],"[4335, 4337, 24073, 41795, 57885]","[4335, 41795, 4337, 24087, 24073]"
49964,"[6366, 15269, 6366, 5895, 6366, 5895]",[5895],"[5895, 6366, 15269]","[5895, 6366, 15269]"
49981,"[64552, 25931, 2807]","[25935, 2807]","[2807, 25931, 64552]","[2807, 64552, 25931]"
49991,"[91921, 20251, 5063, 21742, 5063, 20251, 34927]",[91921],"[5063, 21742, 20251, 34927, 91921]","[91921, 5063, 20251, 21742, 34927]"


In [112]:
df_test_list['recomend_view'] = df_test_list.views.apply(lambda x: get_recomndations(list(map(int, x)), type_rec=0))
df_test_list['recomend_buy'] = df_test_list.views.apply(lambda x: get_recomndations(list(map(int, x)), type_rec=1))

In [113]:
df_test_list.dropna()

,views,buys,recomend_view,recomend_buy
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]","[63, 61, 66, 68, 59]","[63, 68, 69, 70, 66]"
14,"[158, 159, 160, 159, 161, 162]",[162],"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]"
19,"[200, 201, 202, 203, 204]","[201, 205]","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]"
34,"[371, 372, 371]","[371, 373]","[371, 372]","[371, 372]"
40,[422],[422],[422],[422]
...,...,...,...,...
49943,"[2859, 2854, 88887, 96997, 4439, 28645, 99975,...","[24907, 102691, 18496, 4333]","[4335, 4333, 24907, 606, 2854]","[4335, 4333, 606, 2859, 2854]"
49944,"[77655, 23249, 1306, 47450, 26157, 58205, 4745...","[58205, 3111, 69482]","[1262, 16237, 7318, 1261, 1306]","[1262, 1306, 13520, 77655, 23249]"
49945,"[60538, 44430, 66252, 44430, 60538, 66251]","[66252, 44430]","[60538, 44430, 66251, 66252]","[60538, 44430, 66252, 66251]"
49946,"[49815, 76363]",[49815],"[49815, 76363]","[49815, 76363]"


In [58]:
def recall_k(recomendation, target, k):
    if len(target) == 0: return np.NaN
    recall = 0
    for i in range(min(k, len(recomendation))):
        if recomendation[i] in target: recall += 1
    return recall / len(target)

In [88]:
r = ([158, 162, 160, 159, 161], [162], 5)
recall_k(*r)

1.0

In [69]:
def precision_k(recomendation, target, k):
    if len(target) == 0: return np.NaN
    precision = 0
    for i in range(min(k, len(recomendation))):
        if recomendation[i] in target: precision += 1
    return precision / k

In [71]:
precision_k([4335, 4333], [24907, 102691, 18496, 4333, 201, 205], 5)

0.2

In [115]:
r5 = df_list.dropna().apply(lambda x: recall_k(list(map(int, x[2])), list(map(int, x[1])), 5), axis=1).mean()
r1 = df_list.dropna().apply(lambda x: recall_k(list(map(int, x[2])), list(map(int, x[1])), 1), axis=1).mean()
p5 = df_list.dropna().apply(lambda x: precision_k(list(map(int, x[2])), list(map(int, x[1])), 5), axis=1).mean()
p1 = df_list.dropna().apply(lambda x: precision_k(list(map(int, x[2])), list(map(int, x[1])), 1), axis=1).mean()

print(f'{r1:.2f} {p1:.2f} {r5:.2f} {p5:.2f}')

0.44 0.51 0.82 0.21


In [106]:
r5 = df_test_list.dropna().apply(lambda x: recall_k(list(map(int, x[2])), list(map(int, x[1])), 5), axis=1).mean()
r1 = df_test_list.dropna().apply(lambda x: recall_k(list(map(int, x[2])), list(map(int, x[1])), 1), axis=1).mean()
p5 = df_test_list.dropna().apply(lambda x: precision_k(list(map(int, x[2])), list(map(int, x[1])), 5), axis=1).mean()
p1 = df_test_list.dropna().apply(lambda x: precision_k(list(map(int, x[2])), list(map(int, x[1])), 1), axis=1).mean()

print(f'{r1:.2f} {p1:.2f} {r5:.2f} {p5:.2f}')

0.41 0.48 0.80 0.20


In [116]:
r5 = df_list.dropna().apply(lambda x: recall_k(list(map(int, x[3])), list(map(int, x[1])), 5), axis=1).mean()
r1 = df_list.dropna().apply(lambda x: recall_k(list(map(int, x[3])), list(map(int, x[1])), 1), axis=1).mean()
p5 = df_list.dropna().apply(lambda x: precision_k(list(map(int, x[3])), list(map(int, x[1])), 5), axis=1).mean()
p1 = df_list.dropna().apply(lambda x: precision_k(list(map(int, x[3])), list(map(int, x[1])), 1), axis=1).mean()

print(f'{r1:.2f} {p1:.2f} {r5:.2f} {p5:.2f}')

0.68 0.79 0.93 0.25


In [114]:
r5 = df_test_list.dropna().apply(lambda x: recall_k(list(map(int, x[3])), list(map(int, x[1])), 5), axis=1).mean()
r1 = df_test_list.dropna().apply(lambda x: recall_k(list(map(int, x[3])), list(map(int, x[1])), 1), axis=1).mean()
p5 = df_test_list.dropna().apply(lambda x: precision_k(list(map(int, x[3])), list(map(int, x[1])), 5), axis=1).mean()
p1 = df_test_list.dropna().apply(lambda x: precision_k(list(map(int, x[3])), list(map(int, x[1])), 1), axis=1).mean()

print(f'{r1:.2f} {p1:.2f} {r5:.2f} {p5:.2f}')

0.46 0.53 0.82 0.21


In [100]:
df_test_list.dropna().apply(lambda x: recall_k(list(map(int, x[3])), list(map(int, x[1])), 5), axis=1)

7        0.50
14       1.00
19       0.00
34       0.00
40       1.00
         ... 
49943    0.25
49944    0.00
49945    0.00
49946    0.00
49985    1.00
Length: 3665, dtype: float64